In [4]:
import requests
import json
import time
import os

In [12]:
YELP_API_KEY = os.getenv("YELP_API_KEY")

HEADERS = {"Authorization": f"Bearer {YELP_API_KEY}"}

In [14]:
with open("restaurants.json", "r") as f:
    restaurants = json.load(f)

In [19]:
def get_full_address(restaurant):
    try:
        query_params = {
            "term": restaurant["name"],
            "location": "San Diego, CA",
            "limit": 1,
        }
        response = requests.get(
            "https://api.yelp.com/v3/businesses/search",
            headers=HEADERS,
            params=query_params
        )
        response.raise_for_status()
        businesses = response.json().get("businesses", [])
        if not businesses:
            print(f"No match found for: {restaurant['name']}")
            return restaurant

        business_id = businesses[0]["id"]
        details = requests.get(
            f"https://api.yelp.com/v3/businesses/{business_id}",
            headers=HEADERS
        )
        details.raise_for_status()
        full_address = ", ".join(details.json()["location"]["display_address"])
        restaurant["location"] = full_address
        return restaurant
    except Exception as e:
        print(f"Error for {restaurant['name']}: {e}")
        return restaurant


In [20]:
updated_restaurants = []
for r in restaurants:
    enriched = get_full_address(r)
    updated_restaurants.append(enriched)
    time.sleep(0.5) # to avoid hitting rate limits


No match found for: Chungchun Ssal Hot Dog
No match found for: Kjodo BBQ


In [22]:
with open("restaurants.json", "w") as f:
    json.dump(updated_restaurants, f, indent=2)